# Protein Synthesis Simulation

Load dataset:

In [1]:
from HumanGenomeDataset.load_dataset import load_dataset

data_df = load_dataset('dna_protein_coding_sequences')
data_df.head(2)

c:\Users\giuli\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,ID,sequence,category
0,NM_001368254.1,GCTGAGCTGAGCTGGGGCGCAGCCGCCTGTCTGCACCGGCAGCACC...,NM
1,NM_001350977.1,TCCACAACTGAAACATCCACTTCTGAACACCATGTCCTACTACAGC...,NM


### Simulation

In [2]:
from src.simulation import ProteinSinthesisProcess

Load parameters:

In [3]:
import json

parameters_path = 'data/parameters.json'
parameters = json.load(open(parameters_path, 'r'))

In [4]:
from itertools import product

parameters_sets = list(product(*parameters.values()))

Start simulation:

In [5]:
for i, set in enumerate(parameters_sets):
    ps = ProteinSinthesisProcess(data_df, *set[1:])
    ps.run(simulation_time=set[0]) # run the simulation

    ps.save_process(folder_test_name=f'test_set{i}') # save the process
    
    # save the parameters
    with open(f'results/test_set{i}/parameters.json', 'w') as f:
        json.dump({k: v for k, v in zip(parameters.keys(), set)}, f)
    
    print(f'Finished test set {i+1}/{len(parameters_sets)}\n')

Simulation environment initialized, time unit: 0.0001 second.
Simulation started
End simulation: 308 proteins synthesized from 155 DNA sequences.
Process saved.
Finished test set 0/18

Simulation environment initialized, time unit: 0.0001 second.
Simulation started
End simulation: 308 proteins synthesized from 155 DNA sequences.
Process saved.
Finished test set 1/18

Simulation environment initialized, time unit: 0.0001 second.
Simulation started
